Clase 5
====

Objetivo:

* Introducción a cobrapy

La documentación oficial de cobra se encuentra disponible [aquí](http://cobrapy.readthedocs.io/en/0.4.0/getting_started.html). Hoy veremos los comandos escenciales de cobra y construiremos nuestra propia red metábolica.

Comandos básicos
--------------

Comezaremos cargado cobra y determinando que version está instalada (esto es útil para consultar la documentación adecuada; diferentes versiones de cobra tienen diferentes documentaciones)

In [ ]:
import cobra
cobra.__version__

A modo de ejemplo importaremos la red metabólica de Escherichia coli

In [ ]:
import cobra.test

# "ecoli" and "salmonella" are also valid arguments
model = cobra.test.create_test_model("ecoli")

Veamos cuantas reacciones y metabolitos tiene "ecoli"

In [ ]:
print(len(model.reactions))
print(len(model.metabolites))

Podemos inspeccionar reacciones individualmente. Por ejemplo, para ver la reacción 30:

In [ ]:
print model.reactions[29]

Ver las primeras 10 reacciones:

In [ ]:
for i in range(10):
    print model.reactions[i]

También podemos identificar una reacción por su nombre:

In [ ]:
pgi = model.reactions.get_by_id("PGI")
print pgi
print pgi.reaction
print pgi.name


print(pgi.lower_bound, "< pgi <", pgi.upper_bound)

Podemos verificar el balance de masa de una reacción:

In [ ]:
# Balance de masa original
print pgi.check_mass_balance()

# Veamos que pasa con el balance de masa si ahora agregamos un metabolito a la reacción
pgi.add_metabolites({model.metabolites.get_by_id("h_c"): -1})
pgi.reaction
print pgi.check_mass_balance()

Tambien podemos ver cuales son las reacciones en las que un determinado metabólito está involucrado:

In [ ]:
for rxn in model.metabolites.get_by_id("atp_c").reactions:
    print rxn, rxn.reaction

Simulación de flujos metabólicos
================================

Para simular los flujos metabólicos optimizacmos los flujos de la red metabólica:

In [ ]:
print model.optimize()

Los resultados se pueder revisar:

In [ ]:
print model.solution.status
print model.solution.f
print model.summary()

Para inspeccionar como la función objetivo (en este caso:"Ec_biomass_iJO1366_core_53p95M")

In [ ]:
biomass = model.reactions.get_by_id("Ec_biomass_iJO1366_core_53p95M")
print biomass.reaction


Planos de fase
--------------

Los planos de fase son útiles cuando se quiere evaluar como cambios en las variables (tipicamente flujos de entrada) afectan el valor de la función objetivo (tipicamente producción de biomasa).

Veamos una representación gráfica usando otra red de ejemplo (llamada model2):

In [ ]:
%matplotlib inline
from time import time

from cobra.flux_analysis import calculate_phenotype_phase_plane

model2 = cobra.test.create_test_model("textbook")
data = calculate_phenotype_phase_plane(model2, "EX_glc__D_e", "EX_o2_e")
data.plot_matplotlib();

Ejercicio
---------

* Realizar este gráfico para la red de ecoli variando el consumo de glucosa y oxigeno.

Nota: deberas identificar las reacciones asociados al consumo de glucosa y oxigeno (pista: usa el comando `model.summary()` ).

Building a model
=========

Ahora construiremos nuestra propia red metaolica usando como ejemplo la red metanogénica que vimos en la clase pasada:

![Image](https://raw.githubusercontent.com/modcommet/Clases/master/metanogenesis.png)

In [ ]:
from cobra import Model, Reaction, Metabolite
cobra_model = Model('primordialMetabolism')

# Ractions
E1 = Reaction('E1')
E1.name = 'Exchange reaction of CO2'
E1.subsystem = 'Entrance'
E1.lower_bound = 10.  # This is the default
E1.upper_bound = 1000.  # This is the default
E1.objective_coefficient = 0. # this is the default

E2 = Reaction('E2')
E2.name = 'Exchange reaction of H2'
E2.subsystem = 'Entrance'
E2.lower_bound = 0.  # This is the default
E2.upper_bound = 1000.  # This is the default
E2.objective_coefficient = 0. # this is the default

# Agrega el resto de las reacciones

# Metabolites
CO2 = Metabolite('CO2', formula='CO2',name='carbon dioxide',compartment='c')
H2 = Metabolite('H2', formula='H2',name='H2',compartment='c')

# Agrega el resto de los metabolitos

# Adding metabolites to reaction
E1.add_metabolites({CO2: 1.0})
E2.add_metabolites({H2: 2.0})

# ya sabes que hacer :P

# Add reactin to model
cobra_model.add_reaction(E1)
cobra_model.add_reaction(E2)

# lo mismo aquí

Puedes revisar tu modelo con los siguientes comandos

In [ ]:
# Iterate through the the objects in the model
print("Reactions")
print("---------")
for x in cobra_model.reactions:
    print("%s : %s" % (x.id, x.reaction))
print("Metabolites")
print("-----------")
for x in cobra_model.metabolites:
    print('%s : %s' % (x.id, x.formula))

Ejercicio
---------

* Simula los flujos de esta red metabólica.
    